In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import pprint

import folium
from folium.plugins import MarkerCluster
import googlemaps
import geopandas as gpd

import requests
import json
import time

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
bus_time_table = pd.read_csv('./datasets/bus_time_table.csv', encoding='cp949', sep=',', parse_dates=['기준_날짜'])
bus_time_table.isna().sum()

기준_날짜            0
행정동_ID           0
버스_승객_수        424
버스_승객_수_00시    424
버스_승객_수_01시    424
버스_승객_수_02시    424
버스_승객_수_03시    424
버스_승객_수_04시    424
버스_승객_수_05시    424
버스_승객_수_06시    424
버스_승객_수_07시    424
버스_승객_수_08시    424
버스_승객_수_09시    424
버스_승객_수_10시    424
버스_승객_수_11시    424
버스_승객_수_12시    424
버스_승객_수_13시    424
버스_승객_수_14시    424
버스_승객_수_15시    424
버스_승객_수_16시    424
버스_승객_수_17시    424
버스_승객_수_18시    424
버스_승객_수_19시    424
버스_승객_수_20시    424
버스_승객_수_21시    424
버스_승객_수_22시    424
버스_승객_수_23시    424
dtype: int64

In [3]:
# 2022-12-18 삭제
del_idx = bus_time_table[bus_time_table.isna().values == True].index
bus_time_table = bus_time_table.drop(del_idx, axis=0)
bus_time_table.isna().sum()

기준_날짜          0
행정동_ID         0
버스_승객_수        0
버스_승객_수_00시    0
버스_승객_수_01시    0
버스_승객_수_02시    0
버스_승객_수_03시    0
버스_승객_수_04시    0
버스_승객_수_05시    0
버스_승객_수_06시    0
버스_승객_수_07시    0
버스_승객_수_08시    0
버스_승객_수_09시    0
버스_승객_수_10시    0
버스_승객_수_11시    0
버스_승객_수_12시    0
버스_승객_수_13시    0
버스_승객_수_14시    0
버스_승객_수_15시    0
버스_승객_수_16시    0
버스_승객_수_17시    0
버스_승객_수_18시    0
버스_승객_수_19시    0
버스_승객_수_20시    0
버스_승객_수_21시    0
버스_승객_수_22시    0
버스_승객_수_23시    0
dtype: int64

In [4]:
# 타입변경
bus_time_table[['버스_승객_수', '버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']] = bus_time_table[['버스_승객_수', '버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].astype('int')
bus_time_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173416 entries, 0 to 173839
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   기준_날짜        173416 non-null  datetime64[ns]
 1   행정동_ID       173416 non-null  int64         
 2   버스_승객_수      173416 non-null  int32         
 3   버스_승객_수_00시  173416 non-null  int32         
 4   버스_승객_수_01시  173416 non-null  int32         
 5   버스_승객_수_02시  173416 non-null  int32         
 6   버스_승객_수_03시  173416 non-null  int32         
 7   버스_승객_수_04시  173416 non-null  int32         
 8   버스_승객_수_05시  173416 non-null  int32         
 9   버스_승객_수_06시  173416 non-null  int32         
 10  버스_승객_수_07시  173416 non-null  int32         
 11  버스_승객_수_08시  173416 non-null  int32         
 12  버스_승객_수_09시  173416 non-null  int32         
 13  버스_승객_수_10시  173416 non-null  int32         
 14  버스_승객_수_11시  173416 non-null  int32         
 15  버스_승객_수_12시  173416 non-null  int3

In [5]:
# 승객수 시간별 총합이 버스 승객수가 맞는지 확인
bus_time_table[bus_time_table[['버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].sum(axis=1) != bus_time_table['버스_승객_수']]

,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시


In [6]:
# 중구 22년 3월만
bus_time_table_3month = bus_time_table[(bus_time_table['행정동_ID']//100==11020)&(bus_time_table['기준_날짜'].dt.month==3)].sort_values(['기준_날짜'])
bus_time_table_3month.head()

,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시
149193,2022-03-01,1102055,9767,27,8,7,11,20,52,86,...,925,905,888,812,714,604,549,623,510,170
148875,2022-03-01,1102052,15399,54,17,10,10,43,111,135,...,1474,2001,1663,1683,1312,993,1093,608,540,174
148979,2022-03-01,1102073,786,5,0,0,0,0,1,16,...,60,49,35,54,53,52,29,49,39,19
148980,2022-03-01,1102068,2367,15,3,3,2,4,36,52,...,155,175,170,181,150,124,130,104,132,39
148981,2022-03-01,1102059,7777,94,87,64,117,171,322,122,...,556,618,689,726,569,411,374,370,354,170


In [7]:
bus_total = bus_time_table_3month.groupby('행정동_ID')[['버스_승객_수']].sum().reset_index()
bus_total

,행정동_ID,버스_승객_수
0,1102052,688691
1,1102054,705555
2,1102055,496049
3,1102057,85316
4,1102058,83509
5,1102059,302658
6,1102060,97719
7,1102065,148232
8,1102067,91718
9,1102068,113756


In [8]:
state_geo = gpd.read_file('./datasets/seoul_geo_dong.geojson')
state_geo.head(2)

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
0,1,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구 사직동,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,2,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구 삼청동,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."


In [9]:
junggu_geo = state_geo[ state_geo['adm_cd'].astype('int')//100 == 11020 ]
junggu_geo

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
17,18,서울특별시 중구 소공동,1102052,1114052000,11140,11,서울특별시,중구 소공동,중구,11020520,"MULTIPOLYGON (((126.98109 37.56180, 126.98050 ..."
18,19,서울특별시 중구 회현동,1102054,1114054000,11140,11,서울특별시,중구 회현동,중구,11020540,"MULTIPOLYGON (((126.98536 37.55424, 126.98543 ..."
19,20,서울특별시 중구 명동,1102055,1114055000,11140,11,서울특별시,중구 명동,중구,11020550,"MULTIPOLYGON (((126.97703 37.56919, 126.97758 ..."
20,21,서울특별시 중구 필동,1102057,1114057000,11140,11,서울특별시,중구 필동,중구,11020570,"MULTIPOLYGON (((126.99347 37.56300, 126.99339 ..."
21,22,서울특별시 중구 장충동,1102058,1114058000,11140,11,서울특별시,중구 장충동,중구,11020580,"MULTIPOLYGON (((127.00957 37.56303, 127.00936 ..."
22,23,서울특별시 중구 광희동,1102059,1114059000,11140,11,서울특별시,중구 광희동,중구,11020590,"MULTIPOLYGON (((127.00957 37.56303, 127.00670 ..."
23,24,서울특별시 중구 을지로동,1102060,1114060500,11140,11,서울특별시,중구 을지로동,중구,11020600,"MULTIPOLYGON (((126.99347 37.56300, 126.99143 ..."
24,25,서울특별시 중구 신당5동,1102065,1114065000,11140,11,서울특별시,중구 신당5동,중구,11020650,"MULTIPOLYGON (((127.02362 37.56517, 127.02671 ..."
25,26,서울특별시 중구 황학동,1102067,1114067000,11140,11,서울특별시,중구 황학동,중구,11020670,"MULTIPOLYGON (((127.02362 37.56517, 127.02059 ..."
26,27,서울특별시 중구 중림동,1102068,1114068000,11140,11,서울특별시,중구 중림동,중구,11020680,"MULTIPOLYGON (((126.97140 37.55960, 126.97057 ..."


In [10]:
here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)
m

In [11]:
state_geo

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
0,1,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구 사직동,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,2,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구 삼청동,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,3,서울특별시 종로구 부암동,1101055,1111055000,11110,11,서울특별시,종로구 부암동,종로구,11010550,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,4,서울특별시 종로구 평창동,1101056,1111056000,11110,11,서울특별시,종로구 평창동,종로구,11010560,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,5,서울특별시 종로구 무악동,1101057,1111057000,11110,11,서울특별시,종로구 무악동,종로구,11010570,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...,...,...,...,...,...
3511,3456,세종특별자치시 세종시 반곡동,2901075,3611055600,36110,36,세종특별자치시,세종시 반곡동,세종시,29010513,"MULTIPOLYGON (((127.29641 36.54349, 127.30004 ..."
3512,3492,세종특별자치시 세종시 새롬동,2901073,3611051500,36110,36,세종특별자치시,세종시 새롬동,세종시,29010511,"MULTIPOLYGON (((127.23907 36.48808, 127.24342 ..."
3513,1599,경기도 광주시 오포2동,3125061,4161059000,41610,41,경기도,광주시 오포2동,광주시,31250512,"MULTIPOLYGON (((127.23876 37.35970, 127.23997 ..."
3514,1599,경기도 광주시 오포1동,3125060,4161058000,41610,41,경기도,광주시 오포1동,광주시,31250511,"MULTIPOLYGON (((127.19171 37.34718, 127.19221 ..."


In [13]:
state_geo = gpd.read_file('./datasets/seoul_geo_sigugun.json')
seoul_ingu = pd.read_csv("./datasets/주민등록인구_20230222161234.csv", sep='\t', encoding='cp949')

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

m.choropleth(
    geo_data=state_geo,
    data=seoul_ingu,
    columns=['동별', '총계'],
    key_on='feature.properties.name')
m

In [14]:
here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo).add_to(m)

m.choropleth(
    geo_data=junggu_geo,
    data=bus_total,
    columns=['행정동_ID', '버스_승객_수'],
    key_on='feature.properties.adm_cd')
m